## Import Libraries and Declare API Key

In [1]:
import requests
import pandas as pd
import numpy as np
import signal

In [2]:
API_KEY = {'X-API-key': 'RPCJTKLW'}

## Current Heat Trading Algorithm

### Basic Buy and Sell Functions

In [18]:
shutdown = False

class ApiException(Exception):
    pass

def signal_handler(signum, frame):
    global shutdown
    signal.signal(signal.SIGINT, signal.SIG_DFL)
    shutdown = True

def get_tick(session):
    resp = session.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick']
    raise ApiException('Authorization error. Please check API key.')

def ticker_bid_ask(session, ticker):
    payload = {'ticker': ticker}
    resp = session.get('http://localhost:9999/v1/securities/book', params = payload)
    if resp.ok:
        book = resp.json()
        return book['bids'][0]['price'], book['asks'][0]['price']
    raise ApiException('Authorization error. Please check API key.')

def buy(session, ticker, type, q, price = None):
    if type == 'MARKET':
        resp = session.post('http://localhost:9999/v1/orders', params = {'ticker': ticker, 'type': 'MARKET', 'quantity': q, 'action': 'BUY'})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
    else:
        resp = session.post('http://localhost:9999/v1/orders',
                            params={'ticker': ticker, 'type': 'LIMIT', 'quantity': q, 'action': 'BUY', 'price': price})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')

def sell(session, ticker, type, q, price = None):
    if type == 'MARKET':
        resp = session.post('http://localhost:9999/v1/orders', params = {'ticker': ticker, 'type': 'MARKET', 'quantity': q, 'action': 'SELL'})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
    else:
        resp = session.post('http://localhost:9999/v1/orders',
                            params={'ticker': ticker, 'type': 'LIMIT', 'quantity': q, 'action': 'SELL', 'price': price})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')

def cancel(session, order_id):
    order_id = order_id # assuming the order to cancel has ID 100
    resp = session.delete('http://localhost:9999/v1/orders/{}'.format(order_id))
    if resp.ok:
        status = resp.json()
        success = status['success']
        print('The order was successfully cancelled?', success)
        
def tender_list(session):
    resp = session.get('http://localhost:9999/v1/tenders')
    if resp.ok:
        lst = resp.json()
        if lst:
            return lst[0]
        else:
            return None
        
def acc_tender(session, tender, price = None):
    if tender['is_fixed_bid'] is True:
        resp = session.post('http://localhost:9999/v1/tenders/{}'.format(tender['tender_id']), 
                           params = {'id': tender['tender_id']})
        if resp.ok:
            print('tender accepted!')
    else:
        resp = session.post('http://localhost:9999/v1/tenders/{}'.format(tender['tender_id']), 
                           params = {'price': price, 'id': tender['tender_id']})
        if resp.ok:
            print('tender accepted!')
      
    
def dec_tender(sesstion, tender):
    resp = session.delete('http://localhost:9999/v1/tenders/{}'.format(tender['tender_id']), 
                           params = {'id': tender['tender_id']})
    if resp.ok:
        print('tender declined!')

## Algorithm Execution Section

In [20]:
signal.signal(signal.SIGINT, signal_handler)

with requests.Session() as s:
    s.headers.update(API_KEY)
    tick = get_tick(s) 
    print(tick)
    buy(s, 'USD', 'MARKET', 1000)
    if tender_list(s):
        acc_tender(s, tender_list(s))

88
The market buy order was submitted and has ID 957
tender accepted!
